# 測試Q矩陣算法

## 結論

GPT不可信
雖然自己的也可以用，但直接用cv2.stereoRectify()省事

In [4]:
import cv2
import numpy as np

In [26]:
def compute_Q_matrix(P1, P2):
    f = P1[0][0]
    cx1 = P1[0][2]
    cy = P1[1][2]
    cx2 = P2[0][2]
    Tx = P2[0][3] / f

    Q = np.array([[1, 0, 0, -cx1], [0, 1, 0, -cy], [0, 0, 0, f],
                 [0, 0, -1 / Tx, (cx1 - cx2) / Tx]])
    return Q


def gpt_compute_Q_matrix(P1, P2):
    # 計算左右相機的投影矩陣
    K1 = P1[:, :3]  # 左相機的相機內部參數矩陣
    K2 = P2[:, :3]  # 右相機的相機內部參數矩陣

    # 左右相機的光心位置
    C1 = -np.linalg.inv(K1) @ P1[:, 3]
    C2 = -np.linalg.inv(K2) @ P2[:, 3]

    # 計算基線向量
    T = C2 - C1

    # 計算 tx
    tx = T[0]

    # 計算Q矩陣的內容
    Q = np.array([[1, 0, 0, -C1[0]],
                  [0, 1, 0, -C1[1]],
                  [0, 0, 0, K1[0, 0]],
                  [0, 0, -1 / tx, 0]])

    return Q

In [28]:
# KITTI
S_02 = np.array([1.392000e+03, 5.120000e+02])
K_02 = np.array([[9.597910e+02, 0.000000e+00, 6.960217e+02], [0.000000e+00,
                9.569251e+02, 2.241806e+02], [0.000000e+00, 0.000000e+00, 1.000000e+00]])
D_02 = np.array([-3.691481e-01, 1.968681e-01,
                1.353473e-03, 5.677587e-04, -6.770705e-02])
R_02 = np.array([[9.999758e-01, -5.267463e-03, -4.552439e-03], [5.251945e-03,
                9.999804e-01, -3.413835e-03], [4.570332e-03, 3.389843e-03, 9.999838e-01]])
T_02 = np.array([5.956621e-02, 2.900141e-04, 2.577209e-03])
S_rect_02 = np.array([1.242000e+03, 3.750000e+02])
R_rect_02 = np.array([[9.998817e-01, 1.511453e-02, -2.841595e-03], [-1.511724e-02,
                     9.998853e-01, -9.338510e-04], [2.827154e-03, 9.766976e-04, 9.999955e-01]])
P_rect_02 = np.array([[7.215377e+02, 0.000000e+00, 6.095593e+02, 4.485728e+01], [0.000000e+00,
                     7.215377e+02, 1.728540e+02, 2.163791e-01], [0.000000e+00, 0.000000e+00, 1.000000e+00, 2.745884e-03]])


S_03 = np.array([1.392000e+03, 5.120000e+02])
K_03 = np.array([[9.037596e+02, 0.000000e+00, 6.957519e+02], [0.000000e+00,
                9.019653e+02, 2.242509e+02], [0.000000e+00, 0.000000e+00, 1.000000e+00]])
D_03 = np.array([-3.639558e-01, 1.788651e-01,
                6.029694e-04, -3.922424e-04, -5.382460e-02])
R_03 = np.array([[9.995599e-01, 1.699522e-02, -2.431313e-02], [-1.704422e-02,
                9.998531e-01, -1.809756e-03], [2.427880e-02, 2.223358e-03, 9.997028e-01]])
T_03 = np.array([-4.731050e-01, 5.551470e-03, -5.250882e-03])
S_rect_03 = np.array([1.242000e+03, 3.750000e+02])
R_rect_03 = np.array([[9.998321e-01, -7.193136e-03, 1.685599e-02], [7.232804e-03,
                     9.999712e-01, -2.293585e-03], [-1.683901e-02, 2.415116e-03, 9.998553e-01]])
P_rect_03 = np.array( [[7.215377e+02, 0.000000e+00, 6.095593e+02, -3.395242e+02], [0.000000e+00, 7.215377e+02, 1.728540e+02, 2.199936e+00], [0.000000e+00, 0.000000e+00, 1.000000e+00, 2.729905e-03]])
print(S_02)
R1, R2, P1, P2, Q, RECT1, RECT2 = cv2.stereoRectify(
    K_02, D_02, K_03, D_03, S_rect_02.astype(int), R_02, T_02, R_03)
# print (out)
# R1,R2,P1,P2,Q
print(f"R1:{R1}")
print(f"R2:{R2}")
print(f"P1:{P1}")
print(f"P2:{P2}")
print(f"Q:{Q}")
print("===== Subtract =====")
print(f"R1:{R1-R_rect_02}")
print(f"R2:{R2-R_rect_03}")
print(f"P1:{P1-P_rect_02}")
print(f"P2:{P2-P_rect_03}")

print(f"Q by stereoRectify()\n{Q}")
Q2=compute_Q_matrix(P_rect_02, P_rect_03)
print(f"Q by compute_Q_matrix\n{Q2}")
loss_ave = sum(Q2.flatten() - Q.flatten()) / 12
print(f"average loss={loss_ave}")
Q3=gpt_compute_Q_matrix(P_rect_02, P_rect_03)
print(f"Q by gpt_compute_Q_matrix\n{Q3}")
loss_ave = sum(Q3.flatten() - Q.flatten()) / 12
print(f"average loss={loss_ave}")

[1392.  512.]
R1:[[ 9.99252395e-01 -2.51884753e-04  3.86599048e-02]
 [ 3.19621061e-04  9.99998425e-01 -1.74593692e-03]
 [-3.86594041e-02  1.75698816e-03  9.99250901e-01]]
R2:[[ 0.99905351  0.00486416  0.04322534]
 [-0.00493989  0.99998644  0.00164539]
 [-0.04321675 -0.00185736  0.99906399]]
P1:[[929.4452       0.         697.43959045   0.        ]
 [  0.         929.4452     236.69963646   0.        ]
 [  0.           0.           1.           0.        ]]
P2:[[929.4452       0.         697.43959045  55.4159788 ]
 [  0.         929.4452     236.69963646   0.        ]
 [  0.           0.           1.           0.        ]]
Q:[[   1.            0.            0.         -697.43959045]
 [   0.            1.            0.         -236.69963646]
 [   0.            0.            0.          929.4452    ]
 [   0.            0.          -16.77215164    0.        ]]
===== Subtract =====
R1:[[-0.00062931 -0.01536641  0.0415015 ]
 [ 0.01543686  0.00011312 -0.00081209]
 [-0.04148656  0.00078029 -0.

In [32]:
# ZED
K_l = np.array([[529.39426391, 0., 480.10883457],
       [0., 529.44103948, 261.89182679],
       [0., 0., 1.]])
D_l = np.array([-0.10605301, 0.1460366, -0.00070588, 0.00127769, -0.08716497])
K_r = np.array([[533.18258885, 0., 483.83887505],
       [0., 533.70326086, 257.89711489],
       [0., 0., 1.]])
D_r = np.array([-0.10031628, 0.12932342, -0.00171209, 0.00359951, -0.06602145])
R = np.array([[9.99999268e-01, -1.46090518e-04, 1.20100622e-03],
     [1.44994116e-04, 9.99999573e-01, 9.12938635e-04],
     [-1.20113908e-03, -9.12763828e-04, 9.99998862e-01]])
T = np.array([-1.26457406,0.01701372,0.08976859])
S = np.array([960, 540])
# S = np.array([1920, 1080])

essential_matrix = [[-3.34517598e-05, -8.97840821e-02, 1.69317460e-02],
                    [8.82495960e-02, -1.16737180e-03, 1.26468043e+00],
                    [-1.71970619e-02, -1.26457103e+00, -1.17491210e-03]]
fundamental_matrix = [[-1.02790155e-08, -2.75863045e-05, 9.98387720e-03],
                      [2.70907718e-05, -3.58326955e-07, 1.92614229e-01],
                      [-9.79915096e-03, -1.93723823e-01, 1.00000000e+00]]

# from stereoRectify()
R1_rec = [[9.97482444e-01, -1.35002093e-02, -6.96169405e-02],
          [1.35324422e-02, 9.99908432e-01, -8.61174009e-06],
          [6.96106820e-02, -9.33497166e-04, 9.97573798e-01]]
R2_rec = [[9.97400076e-01, -1.34191304e-02, -7.08026539e-02],
          [1.33863449e-02, 9.99909959e-01, -9.37546324e-04],
          [7.08088598e-02, -1.26799671e-05, 9.97489902e-01]]
P1_rec = np.array([[531.57215017, 0., 560.89138031, 0.],
          [0., 531.57215017, 257.45361137, 0.],
          [0., 0., 1., 0.]])
P2_rec = np.array([[531.57215017, 0., 560.89138031, -673.96460878],
          [0., 531.57215017, 257.45361137, 0.],
          [0., 0., 1., 0.]])
Q = [[1., 0., 0., -560.89138031],
     [0., 1., 0., -257.45361137],
     [0., 0., 0., 531.57215017],
     [0., 0., 0.78872413, -0.]]

print(S)
R1, R2, P1, P2, Q, RECT1, RECT2 = cv2.stereoRectify(
    K_l, D_l, K_r, D_r, S, R, T)
# print (out)
# R1,R2,P1,P2,Q
print(f"R1:{R1}")
print(f"R2:{R2}")
print(f"P1:{P1}")
print(f"P2:{P2}")
print(f"Q:{Q}")
print("===== Subtract =====")
print(f"R1:{R1-R1_rec}")
print(f"R2:{R2-R2_rec}")
print(f"P1:{P1-P1_rec}")
print(f"P2:{P2-P2_rec}")

print(f"Q by stereoRectify()\n{Q}")
Q2 = compute_Q_matrix(P1_rec, P2_rec)
print(f"Q by compute_Q_matrix\n{Q2}")
loss_ave = sum(abs(Q2.flatten() - Q.flatten())) / 12
print(f"average loss={loss_ave}")
Q3 = gpt_compute_Q_matrix(P1_rec, P2_rec)
print(f"Q by gpt_compute_Q_matrix\n{Q3}")
loss_ave = sum(abs(Q3.flatten() - Q.flatten())) / 12
print(f"average loss={loss_ave}")

[960 540]
R1:[[ 9.97482444e-01 -1.35002104e-02 -6.96169397e-02]
 [ 1.35324433e-02  9.99908432e-01 -8.61177300e-06]
 [ 6.96106813e-02 -9.33497200e-04  9.97573798e-01]]
R2:[[ 9.97400076e-01 -1.34191315e-02 -7.08026531e-02]
 [ 1.33863460e-02  9.99909959e-01 -9.37546358e-04]
 [ 7.08088590e-02 -1.26800007e-05  9.97489902e-01]]
P1:[[531.57215017   0.         560.89138031   0.        ]
 [  0.         531.57215017 257.45361137   0.        ]
 [  0.           0.           1.           0.        ]]
P2:[[ 531.57215017    0.          560.89138031 -673.96460901]
 [   0.          531.57215017  257.45361137    0.        ]
 [   0.            0.            1.            0.        ]]
Q:[[   1.            0.            0.         -560.89138031]
 [   0.            1.            0.         -257.45361137]
 [   0.            0.            0.          531.57215017]
 [   0.            0.            0.78872413   -0.        ]]
===== Subtract =====
R1:[[-2.83595369e-11 -1.07150219e-09  8.35229053e-10]
 [ 1.1467822

In [71]:
from pathlib import Path
def get_KITTI_Q_matrix(name: str) -> np.ndarray:
    KITTI_path = Path(
        r'E:\datasets\KITTI_sceneflow\calibration_files\training\calib_cam_to_cam')
    calib_path = KITTI_path / (name + ".txt")
    calib = calib_path.read_text()
    # with calib_path.open('r') as f:
    #     calib = f.readlines()
    calib=calib.split('\n')
    # print(len(calib))
    # print(calib)

    K_02 = np.array(calib[19].strip().split()[1:]
                    ).astype(float).reshape((3, 3))
    D_02 = np.array(calib[20].strip().split()[1:]
                    ).astype(float).flatten()
    R_02 = np.array(calib[21].strip().split()[1:]
                    ).astype(float).reshape((3, 3))
    T_02 = np.array(calib[22].strip().split()[1:]
                    ).astype(float).flatten()
    S_rect_02 = np.array(calib[23].strip().split()[
                         1:]).astype(float).flatten()
    R_rect_02 = np.array(calib[24].strip().split()[
                         1:]).astype(float).reshape((3, 3))
    P_rect_02 = np.array(calib[25].strip().split()[1:]).astype(float).reshape((3, 4))
    K_03 = np.array(calib[27].strip().split()[1:]
                    ).astype(float).reshape((3, 3))
    D_03 = np.array(calib[28].strip().split()[1:]
                    ).astype(float).flatten()
    R_03 = np.array(calib[29].strip().split()[1:]
                    ).astype(float).reshape((3, 3))
    R_rect_03 = np.array(calib[32].strip().split()[
                         1:]).astype(float).reshape((3, 3))
    P_rect_03 = np.array(calib[33].strip().split(
        )[1:]).astype(float).reshape((3, 4))

    print(f"S_rect_02: {S_rect_02}")
    print(f"K_02: {K_02}")
    print(f"D_02: {D_02}")
    print(f"R_02: {R_02}")
    print(f"T_02: {T_02}")
    print(f"K_03: {K_03}")
    print(f"D_03: {D_03}")
    print(f"R_03: {R_03}")

    P_rect_02 = np.array([float(x)
                      for x in calib[25].strip().split()[1:]]).reshape((3, 4))
    P_right = np.array(
        [float(x) for x in calib[33].strip().split()[1:]]).reshape((3, 4))

    R1, R2, P1, P2, Q, RECT1, RECT2 = cv2.stereoRectify(
        K_02, D_02, K_03, D_03, S_rect_02.astype(int), R_02, T_02, R_03)
    print(f"Q: {Q}")
    P_rect_02_err = sum(abs(P1.flatten() - P_rect_02.flatten())) / 12
    P_rect_03_err = sum(abs(P2.flatten() - P_rect_03.flatten())) / 12
    print(f"P_rect_02_err: {P_rect_02_err}")
    print(f"P_rect_03_err: {P_rect_03_err}")
    return Q


get_KITTI_Q_matrix("000000")

S_rect_02: [1242.  375.]
K_02: [[959.791    0.     696.0217]
 [  0.     956.9251 224.1806]
 [  0.       0.       1.    ]]
D_02: [-0.3691481   0.1968681   0.00135347  0.00056776 -0.06770705]
R_02: [[ 0.9999758  -0.00526746 -0.00455244]
 [ 0.00525195  0.9999804  -0.00341384]
 [ 0.00457033  0.00338984  0.9999838 ]]
T_02: [0.05956621 0.00029001 0.00257721]
K_03: [[903.7596   0.     695.7519]
 [  0.     901.9653 224.2509]
 [  0.       0.       1.    ]]
D_03: [-0.3639558   0.1788651   0.00060297 -0.00039224 -0.0538246 ]
R_03: [[ 0.9995599   0.01699522 -0.02431313]
 [-0.01704422  0.9998531  -0.00180976]
 [ 0.0242788   0.00222336  0.9997028 ]]
Q: [[   1.            0.            0.         -697.43959045]
 [   0.            1.            0.         -236.69963646]
 [   0.            0.            0.          929.4452    ]
 [   0.            0.          -16.77215164    0.        ]]
P_rect_02_err: 51.05144432478769
P_rect_03_err: 80.3903143016223


array([[   1.        ,    0.        ,    0.        , -697.43959045],
       [   0.        ,    1.        ,    0.        , -236.69963646],
       [   0.        ,    0.        ,    0.        ,  929.4452    ],
       [   0.        ,    0.        ,  -16.77215164,    0.        ]])